<a href="https://colab.research.google.com/github/joelwwiggins/car_ml/blob/main/Copy_of_notebook_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
import numpy as np
import tensorflow as tf
import pandas as pd

In [45]:
#import csv from path
#drop first row and column
df1 = pd.read_csv('CSVLog_20221218_084434.csv', skiprows=1)


In [46]:
#drop first column
df1_col=df1.iloc[:,1:]
df=df1_col

In [47]:
#drop rows where the vehicle speed in column 2 is <1
df_high_speed=df[df.iloc[:,2]>1]
#drop duplicate rows
df_duplicate=df_high_speed.drop_duplicates()
#how many duplicate rows were dropped
print(len(df_high_speed)-len(df_duplicate))

69007


In [49]:
#convert to numpy array
df_np=df_duplicate.to_numpy()

#train test split
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(df_np, test_size=0.2, random_state=42)

#normalize the x_train
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

#reshape the data to 3d
X_train_scaled = X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1],1))

#use a 1dconv model to predict the next value
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D 
from keras.layers import Dropout
from keras.layers import LSTM
#import resnet 50 model

from keras.preprocessing import image
from keras.models import Model


model_shape = X_train_scaled.shape



nn_model = tf.keras.models.Sequential()
nn_model.build(input_shape=model_shape)

nn_model.add(tf.keras.layers.Conv1D(512,3,activation="relu",padding='same',input_shape=model_shape[1:]))

nn_model.add(tf.keras.layers.MaxPool1D(pool_size=2,data_format='channels_last',padding='same'))
nn_model.add(tf.keras.layers.Conv1D(256,3,activation="relu",padding='same'))
nn_model.add(tf.keras.layers.MaxPool1D(pool_size=2,data_format='channels_last',padding='same'))
nn_model.add(tf.keras.layers.Conv1D(128,3,activation="relu",padding='same'))
nn_model.add(tf.keras.layers.MaxPool1D(pool_size=2,data_format='channels_last',padding='same'))
nn_model.add(tf.keras.layers.Conv1D(64,3,activation="relu",padding='same'))
nn_model.add(tf.keras.layers.MaxPool1D(pool_size=2,data_format='channels_last',padding='same'))
nn_model.add(tf.keras.layers.Conv1D(32,3,activation="relu",padding='same'))
nn_model.add(tf.keras.layers.MaxPool1D(pool_size=2,data_format='channels_last',padding='same'))
nn_model.add(tf.keras.layers.Conv1D(16,3,activation="relu",padding='same'))
nn_model.add(tf.keras.layers.MaxPool1D(pool_size=2,data_format='channels_last',padding='same'))
nn_model.add(tf.keras.layers.Conv1D(8,3,activation="relu",padding='same'))
nn_model.add(tf.keras.layers.MaxPool1D(pool_size=2,data_format='channels_last',padding='same'))
nn_model.add(tf.keras.layers.Conv1D(4,3,activation="relu",padding='same'))
nn_model.add(tf.keras.layers.MaxPool1D(pool_size=2,data_format='channels_last',padding='same'))


nn_model.add(tf.keras.layers.Flatten())
nn_model.add(tf.keras.layers.Dense(units=model_shape[1], activation='linear'))




nn_model.compile(loss='mse', optimizer='adam', metrics=['mae','mse'])
es = tf.keras.callbacks.EarlyStopping(monitor='val_mae',patience=10)
# checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath ="/content/sample_data")
tbCallBack = tf.keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=1, write_graph=True, write_images=True)
cb_list = [es, tbCallBack]
nn_model.fit(X_train, X_train, epochs=30,validation_split=.1,shuffle=True,callbacks=cb_list)
#normalize the x_test
X_test_scaled = scaler.fit_transform(X_test)
nn_model.evaluate(X_test_scaled, X_test_scaled)



nn_model.summary()

Epoch 1/30
465/465 [==============================] - 30s 62ms/step - loss: 2602933.5000 - mae: 233.9730 - mse: 2602933.5000 - val_loss: 24311.1230 - val_mae: 44.3508 - val_mse: 24311.1230
Epoch 2/30
465/465 [==============================] - 28s 61ms/step - loss: 25113.6016 - mae: 47.2324 - mse: 25113.6016 - val_loss: 28240.8027 - val_mae: 62.2017 - val_mse: 28240.8027
Epoch 3/30
465/465 [==============================] - 28s 61ms/step - loss: 24909.5020 - mae: 45.6058 - mse: 24909.5020 - val_loss: 24925.0117 - val_mae: 46.7489 - val_mse: 24925.0117
Epoch 4/30
465/465 [==============================] - 28s 61ms/step - loss: 24668.0449 - mae: 45.3409 - mse: 24668.0449 - val_loss: 30555.7051 - val_mae: 74.7065 - val_mse: 30555.7051
Epoch 5/30
465/465 [==============================] - 28s 60ms/step - loss: 24751.4180 - mae: 45.6210 - mse: 24751.4180 - val_loss: 24236.2109 - val_mae: 45.5178 - val_mse: 24236.2109
Epoch 6/30
465/465 [==============================] - 29s 63ms/step - loss:

KeyboardInterrupt: 

In [41]:
#save the model
nn_model.save('model.h5')

In [42]:
import plotly.express as px
#reshape X_train_scaled ot 2d
X_train_scaled = X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1]))


reconstructions_test = nn_model.predict(X_train_scaled)
test_loss = tf.keras.losses.mae(reconstructions_test, X_train_scaled)


df_loss_test = pd.DataFrame(test_loss)
fig = px.histogram(df_loss_test, x=df_loss_test.iloc[:,0], nbins=50)
fig.update_layout(yaxis_type="log")
fig.show()

In [43]:
#clean outliers from df and retrain
df_loss= nn_model.predict(df)

#inverse transform df_loss
df_loss_inv = scaler.inverse_transform(df_loss)

inv_scaled_loss=tf.keras.losses.mae(df_loss_inv, df)

#add a column to df with the loss
df['loss']=inv_scaled_loss

#trend the loss
fig = px.line(df, x=df.index, y='loss')
fig.show()






In [ ]:
#clean outliers from df and retrain
df_loss= nn_model.predict(df_duplicate)

#inverse transform df_loss
df_loss_inv = scaler.inverse_transform(df_loss)

inv_scaled_loss=tf.keras.losses.mae(df_loss_inv, df_duplicate)

#add a column to df with the loss

df_duplicate['loss']=inv_scaled_loss





#trend the loss
fig = px.line(df_duplicate, x=df_duplicate.index, y='loss')
fig.show()

